In [8]:
## 如何设计模型 
## 1. 考虑具体任务， 分类，回归； 图像，序列类的任务
## 2. 文本 -> embedding  图像 -> 单通道/多通道  得到了可直接用于计算的feature_in
##  feature_in  -> backbone -- > feature_in里提取出有效的feature 
##  图像  -- > CNN -- > stack CNN - > VGG network 
##  文本  -- > embedding -- > RNN 系列  ->  取最后一步的输出 
## 3. output -> head
##  分类 -> 1*10 logits -> softmax -> 概率
##  多分类 -> 1* 10 logits -> 单个维度 做sigmoid / softmax -> 
##  回归 --> fc -> 

## 前向过程至此结束

## 反向传播 -> loss  -> bp ->      

In [9]:
## 任务： 训练一个会做五言绝句的模型
## 输出： 五言绝句序列  输入： 开头 
## one/ many -- > many
## RNN网络的场景

## 文本输入  --> embedding  --> feature_in --- > extract feature (RNN/LSTM/GRU) --> （deep / bi） -> head 

In [24]:
from torch.utils.data import Dataset
# dir(Dataset) # __iter__ 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random

# ==========================================
# 0. 全局配置 (Config)
# ==========================================
class Config:
    vocab_size = 20      # 词表大小 (0-19的数字)
    seq_len = 10         # 序列长度
    embed_dim = 32       # 词向量维度
    hidden_dim = 64      # 隐层维度
    batch_size = 16
    lr = 0.001
    epochs = 20
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ==========================================
# 1. 数据模块 (共用)
# ==========================================
class SharedDataset(Dataset):
    """
    根据 task_mode 返回不同的 (input, target)
    """
    def __init__(self, size=1000, mode='cls'):
        self.size = size
        self.mode = mode
        # 随机生成数据: [size, seq_len]
        self.data = torch.randint(0, Config.vocab_size, (size, Config.seq_len))

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        x = self.data[idx]   #shape (size, Config.seq_len)

        ### many -> one
        if self.mode == 'cls':
            # 任务：分类 (Classification)
            # 规则：如果序列和为偶数，label=0，否则 label=1
            label = 1 if x.sum().item() % 2 != 0 else 0
            return x, torch.tensor(label, dtype=torch.long)

        ## many -> many
        ### 我 爱 自 然 语 言 处 理
        ## 爱 自 然 语 言 处 理  。
        elif self.mode == 'gen':
            # 任务：续写/生成 (Language Modeling)
            # 规则：输入 [x1, x2, x3]，目标 [x2, x3, x4] (错位预测)
            # 这里简单构造：Target 是 Input 循环左移一位
            y = torch.roll(x, -1)
            return x, y

        ## many -> many 
        elif self.mode == 'trans':
            # 任务：翻译 (Seq2Seq)
            # 规则：模拟翻译，这里我们将序列 "倒序" 作为翻译目标
            # Input: [1, 2, 3] -> Target: [3, 2, 1]
            y = torch.flip(x, [0])
            return x, y
        
        else:
            raise ValueError("Unknown mode")

In [12]:
import torch.nn as nn
# help(nn.GRU)

In [ ]:
class MLPClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # MLP 必须把整个序列展平 (Seq_Len * Embed_Dim)

        ## torch构建网络
        self.embedding = nn.Embedding(Config.vocab_size, Config.embed_dim)
        
        #经过embedding 后[batch, seq_len, embed_dim] -> 展平 -> [batch, seq_len * embed_dim]
        self.fc = nn.Sequential(
            ## W (Config.seq_len * Config.embed_dim, 128)
            nn.Linear(Config.seq_len * Config.embed_dim, 128),  ## 全连接   --> batch * 128
            nn.ReLU(), ## 激活
            nn.Linear(128, 2) # 2类 ## 全连接  --> batch *2
         )

    def forward(self, x):
        # x: [batch, seq_len]
        embeds = self.embedding(x) # [batch, seq_len, embed]
        # Flatten: MLP 无法处理变长序列，必须固定长度
        ## reshape, 展平变成[batch, seq_len* embed]
        flat = embeds.view(embeds.size(0), -1) 
        return self.fc(flat)  ## batch * 2

In [ ]:
import torch.nn as nn
# RNN ->h_t+1=tanh(W_ih *x_t + b_ih + W_hh * h_t + b_hh)

In [ ]:
# --- 模型 B: RNN 分类器 (Many-to-One) ---
class RNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(Config.vocab_size, Config.embed_dim)
        # batch_first=True 让输入维度变为 [batch, seq, feature] 
        self.rnn = nn.RNN(Config.embed_dim, Config.hidden_dim, batch_first=True)
        self.fc = nn.Linear(Config.hidden_dim, 2)

    def forward(self, x):
        # x.shape: [batch, seq] 
        embeds = self.embedding(x)
        #embeds.shape: [batch, seq,embed_dim] 
        # out: 每个时间步的输出, h_n: 最后一个时间步的隐状态
        out, h_n = self.rnn(embeds) 
        # 分类任务通常只取最后一个时间步的隐状态 h_n    
        # h_n shape: (num_layers × num_directions, batch_size, hidden_size) [1, batch, hidden] -> squeeze -> [batch, hidden]
        last_hidden = h_n.squeeze(0)
        return self.fc(last_hidden)

In [15]:
# --- 模型 C: RNN 生成器/续写 (Many-to-Many Synced) ---
class RNNGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(Config.vocab_size, Config.embed_dim)
        self.rnn = nn.RNN(Config.embed_dim, Config.hidden_dim, batch_first=True)
        # 输出层要把隐状态映射回词表大小，预测下一个词
        self.fc = nn.Linear(Config.hidden_dim, Config.vocab_size)

    def forward(self, x):
        embeds = self.embedding(x)
        # out shape: [batch, seq_len, hidden]
        out, _ = self.rnn(embeds)
        # 对每一个时间步都进行预测
        prediction = self.fc(out) # [batch, seq_len, vocab_size]
        return prediction

In [16]:
# --- 模型 D: Seq2Seq 翻译器 (Encoder-Decoder) ---
class RNNTranslator(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(Config.vocab_size, Config.embed_dim)
        
        # Encoder: 负责理解输入序列
        self.encoder = nn.RNN(Config.embed_dim, Config.hidden_dim, batch_first=True)
        
        # Decoder: 负责生成输出序列
        self.decoder = nn.RNN(Config.embed_dim, Config.hidden_dim, batch_first=True)
        self.fc = nn.Linear(Config.hidden_dim, Config.vocab_size)

    def forward(self, x, target=None):
        # x: Source [batch, seq_len]
        # target: Target [batch, seq_len] (训练时用于 Teacher Forcing)
        
        batch_size = x.size(0)
        seq_len = x.size(1)
        
        # 1. Encode
        enc_embeds = self.embedding(x)
        _, h_n = self.encoder(enc_embeds) # 获取 Encoder 最后的隐状态
        
        # 2. Decode
        # 解码器的初始隐状态 = 编码器的最终隐状态 (Context Vector)
        decoder_hidden = h_n
        
        # 教学演示简单起见，我们假设 Decoder 的第一个输入是全0或者特定的 Start Token
        # 这里简单构造一个 Start Token (假设为0)
        decoder_input = torch.zeros((batch_size, 1), dtype=torch.long).to(x.device)
        
        outputs = []
        
        for t in range(seq_len):
            dec_embed = self.embedding(decoder_input) # [batch, 1, embed]
            
            # 单步运行 Decoder
            out, decoder_hidden = self.decoder(dec_embed, decoder_hidden)
            step_out = self.fc(out) # [batch, 1, vocab_size]
            outputs.append(step_out)
            
            # 决定下一个输入 (Teacher Forcing vs Autoregressive)
            # 如果提供了 target (训练阶段)，有概率使用真实值作为下一步输入
            if target is not None:
                decoder_input = target[:, t].unsqueeze(1) # Teacher Forcing
            else:
                # 推理阶段：使用自己上一步预测最大概率的词
                top1 = step_out.argmax(2)
                decoder_input = top1
                
        # 拼接所有时间步的输出
        outputs = torch.cat(outputs, dim=1) # [batch, seq_len, vocab_size]
        return outputs

In [ ]:
def train_model(model, mode, epochs=10):
    print(f"\n>>> 开始训练任务: [{mode}] | 模型: {model.__class__.__name__}")
    
    # 构建数据
    dataset = SharedDataset(size=1000, mode=mode)
    loader = DataLoader(dataset, batch_size=Config.batch_size, shuffle=True)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=Config.lr) ## 参数更新的问题
    model.to(Config.device)
    
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total_samples = 0
        
        for x, y in loader:
            x, y = x.to(Config.device), y.to(Config.device)
            optimizer.zero_grad()
            
            if mode == 'cls':
                # 分类任务
                outputs = model(x) # [batch, 2]
                loss = criterion(outputs, y)
                preds = outputs.argmax(dim=1) ## sofmatx -> array -> 取矩阵最大值的索引
                correct += (preds == y).sum().item()
                total_samples += y.size(0)
                
            elif mode == 'gen':
                # 续写任务
                outputs = model(x) # [batch, seq, vocab]
                # CrossEntropy 需要 (N, C) 或 (N, C, d1...)，这里 reshape
                # view() 是一个用于改变张量（Tensor）形状（shape） 的方法，类似于 NumPy 中的 reshape()
                loss = criterion(outputs.view(-1, Config.vocab_size), y.view(-1))
                
            elif mode == 'trans':
                # 翻译任务 (传入 y 用于 Teacher Forcing)
                outputs = model(x, target=y) 
                loss = criterion(outputs.view(-1, Config.vocab_size), y.view(-1))
            
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        # 打印日志
        if (epoch + 1) % 5 == 0:
            avg_loss = total_loss / len(loader)
            if mode == 'cls':
                acc = correct / total_samples
                print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, Acc = {acc:.2%}")
            else:
                print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")

    return model

In [ ]:
if __name__ == "__main__":
    
    # --- 任务 1: 分类 (偶数和检测) ---
    # 对比 MLP 和 RNN
    print("=== 1. 分类任务 (Classification) ===")
    
    # 训练 MLP
    mlp = MLPClassifier()
    train_model(mlp, mode='cls', epochs=10)
    
    # 训练 RNN
    rnn_cls = RNNClassifier()
    train_model(rnn_cls, mode='cls', epochs=10)
    
    print("\n[教学提示]: 对于固定长度的简单序列分类，MLP 和 RNN 都能做好。")
    print("但如果序列长度变长，MLP 参数会爆炸且无法泛化到不同长度，而 RNN 参数共享，更适合序列。")

    # --- 任务 2: 续写 (Generation) ---
    print("\n=== 2. 续写任务 (Generation) ===")
    rnn_gen = RNNGenerator()
    train_model(rnn_gen, mode='gen', epochs=20)
    
    # 测试一下效果
     
    pred_ids = pred.argmax(dim=2)
    print(f"Input:  {test_seq.cpu().numpy()[0]}")
    print(f"Target: {np.roll(test_seq.cpu().numpy()[0], -1)} (左移)")
    print(f"Pred:   {pred_ids.cpu().numpy()[0]}")

    # --- 任务 3: 翻译 (Translation/Seq2Seq) ---
    print("\n=== 3. 翻译任务 (Translation - Reverse Sequence) ===")
    rnn_trans = RNNTranslator()
    train_model(rnn_trans, mode='trans', epochs=20)
    
    # 测试一下效果
    test_seq = torch.randint(0, Config.vocab_size, (1, Config.seq_len)).to(Config.device)
    # 推理时 target=None，不使用 Teacher Forcing
    pred = rnn_trans(test_seq, target=None) 
    pred_ids = pred.argmax(dim=2)
    print(f"Input:   {test_seq.cpu().numpy()[0]}")
    print(f"Target:  {test_seq.cpu().numpy()[0][::-1]} (倒序)")
    print(f"Decoder: {pred_ids.cpu().numpy()[0]}")

=== 1. 分类任务 (Classification) ===

>>> 开始训练任务: [cls] | 模型: MLPClassifier
Epoch 5: Loss = 0.3208, Acc = 91.00%
Epoch 10: Loss = 0.0259, Acc = 100.00%

>>> 开始训练任务: [cls] | 模型: RNNClassifier
Epoch 5: Loss = 0.6733, Acc = 58.40%
Epoch 10: Loss = 0.6249, Acc = 65.70%

[教学提示]: 对于固定长度的简单序列分类，MLP 和 RNN 都能做好。
但如果序列长度变长，MLP 参数会爆炸且无法泛化到不同长度，而 RNN 参数共享，更适合序列。

=== 2. 续写任务 (Generation) ===

>>> 开始训练任务: [gen] | 模型: RNNGenerator
Epoch 5: Loss = 2.9708
Epoch 10: Loss = 2.9454
Epoch 15: Loss = 2.9013
Epoch 20: Loss = 2.8452
Input:  [18  0 15 18 14 10 13  2 16 19]
Target: [ 0 15 18 14 10 13  2 16 19 18] (左移)
Pred:   [15 19  7 14 19  6 18 11  6 13]

=== 3. 翻译任务 (Translation - Reverse Sequence) ===

>>> 开始训练任务: [trans] | 模型: RNNTranslator
Epoch 5: Loss = 2.2162
Epoch 10: Loss = 1.7142
Epoch 15: Loss = 1.4721
Epoch 20: Loss = 1.3175
Input:   [12  6 14  5  9 12  7 19 12  8]
Target:  [ 8 12 19  7 12  9  5 14  6 12] (倒序)
Decoder: [ 8  8 14  4  2  2  8 13  9 19]


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# ==========================================
# 1. 数据准备与预处理
# ==========================================

with open('../poetry.txt', 'r', encoding='utf-8') as f:
    raw_data = f.readlines()
print(raw_data[:10])

['首春:寒随穷律变，春逐鸟声开。初风飘带柳，晚雪间花梅。碧林青旧竹，绿沼翠新苔。芝田初雁去，绮树巧莺来。\n', '初晴落景:晚霞聊自怡，初晴弥可喜。日晃百花色，风动千林翠。池鱼跃不同，园鸟声还异。寄言博通者，知予物外志。\n', '初夏:一朝春夏改，隔夜鸟花迁。阴阳深浅叶，晓夕重轻烟。哢莺犹响殿，横丝正网天。珮高兰影接，绶细草纹连。碧鳞惊棹侧，玄燕舞檐前。何必汾阳处，始复有山泉。\n', '度秋:夏律昨留灰，秋箭今移晷。峨嵋岫初出，洞庭波渐起。桂白发幽岩，菊黄开灞涘。运流方可叹，含毫属微理。\n', '仪鸾殿早秋:寒惊蓟门叶，秋发小山枝。松阴背日转，竹影避风移。提壶菊花岸，高兴芙蓉池。欲知凉气早，巢空燕不窥。\n', '秋日即目:爽气浮丹阙，秋光澹紫宫。衣碎荷疏影，花明菊点丛。袍轻低草露，盖侧舞松风。散岫飘云叶，迷路飞烟鸿。砌冷兰凋佩，闺寒树陨桐。别鹤栖琴里，离猿啼峡中。落野飞星箭，弦虚半月弓。芳菲夕雾起，暮色满房栊。\n', '山阁晚秋:山亭秋色满，岩牖凉风度。疏兰尚染烟，残菊犹承露。古石衣新苔，新巢封古树。历览情无极，咫尺轮光暮。\n', '帝京篇十首:秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉余。连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。惊雁落虚弦，啼猿悲急箭。阅赏诚多美，于兹乃忘倦。鸣笳临乐馆，眺听欢芳节。急管韵朱弦，清歌凝白雪。彩凤肃来仪，玄鹤纷成列。去兹郑卫声，雅音方可悦。芳辰追逸趣，禁苑信多奇。桥形通汉上，峰势接云危。烟霞交隐映，花鸟自参差。何如肆辙迹，万里赏瑶池。飞盖去芳园，兰桡游翠渚。萍间日彩乱，荷处香风举。桂楫满中川，弦歌振长屿。岂必汾河曲，方为欢宴所。落日双阙昏，回舆九重暮。长烟散初碧，皎月澄轻素。搴幌玩琴书，开轩引云雾。斜汉耿层阁，清风摇玉树。欢乐难再逢，芳辰良可惜。玉酒泛云罍，兰殽陈绮席。千钟合尧禹，百兽谐金石。得志重寸阴，忘怀轻尺璧。建章欢赏夕，二八尽妖妍。罗绮昭阳殿，芬芳玳瑁筵。佩移星正动，扇掩月初圆。无劳上悬圃，即此对神仙。以兹游观极，悠然独长想。披卷览前踪，抚躬寻既往。望古茅茨约，瞻今兰殿广。人道恶高危，虚心戒盈荡。奉天竭诚敬，临民思惠养。纳善察忠谏，明科慎刑赏。六五诚难继，四三

In [20]:
import torch
# help(torch.roll)
x = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])
print(torch.roll(x, -1))



tensor([2, 3, 4, 5, 6, 7, 8, 1])


In [ ]:
# help(nn.Embedding)
embedding = nn.Embedding(10, 3)
print(embedding.weight)
input = torch.LongTensor([[1], [9]])
print(embedding(input)) #[1] 和 [9] 对应的embeding

Parameter containing:
tensor([[ 0.2385, -1.8266,  1.7584],
        [ 0.6459, -0.5908,  0.8494],
        [ 0.8723,  1.1961, -0.4590],
        [ 1.7069, -0.3910,  0.9898],
        [-1.6725, -0.5110, -0.2019],
        [ 0.2438,  2.0117, -1.4535],
        [ 0.6208,  1.1696, -0.6816],
        [-0.5261, -0.9986, -0.9125],
        [-0.1360,  0.1364, -0.5567],
        [-1.6452,  0.2102,  0.2038]], requires_grad=True)
tensor([[[ 0.6459, -0.5908,  0.8494]],

        [[-1.6452,  0.2102,  0.2038]]], grad_fn=<EmbeddingBackward0>)


In [25]:
# help(nn.Embedding)

In [ ]:
class TextPipeline:
    def __init__(self, data_list):
        # 1. 清洗数据：这里我们把标题去掉，只保留诗句内容，让模型专注于学写诗
        # 也可以选择保留标题，看你想让模型学什么
        self.sentences = []
        for line in data_list[:10]:
            if ':' in line:
                print(line)
                _, content = line.split(':') # 去掉 "首春:" 保留后面
                self.sentences.append(content)
            else:
                self.sentences.append(line)
        
        # 2. 构建词表 (字 -> ID)
        all_text = "".join(self.sentences)
        self.chars = sorted(list(set(all_text)))
        self.vocab_size = len(self.chars)
        self.char2idx = {c: i for i, c in enumerate(self.chars)}
        self.idx2char = {i: c for i, c in enumerate(self.chars)}
        
        print(f"数据加载完毕，共 {len(self.sentences)} 首诗，词表大小: {self.vocab_size}")

    def text_to_indices(self, text):
        return [self.char2idx[c] for c in text if c in self.char2idx]

    def indices_to_text(self, indices):
        return "".join([self.idx2char[i] for i in indices])

pipeline = TextPipeline(raw_data)

# ==========================================
# 2. 数据集构建 (Sliding Window)
# ==========================================

### 序列
## x0 x1 x2 x3 x4 x5 x6 
## 初始化一个h0
## 0 时刻： h0, x0 -> RNN -> h1, o0  --> x1
## 1 时刻  h1, x1 -> RNN -> h2, o1   --> x2
## 2 时刻  h2, x2 -> RNN -> h3, o2   --> x3

## torch.roll

class PoetryDataset(Dataset):
    def __init__(self, pipeline, seq_len=6):
        self.inputs = []
        self.targets = []
        
        # 滑动窗口构造数据
        # 输入: "寒随穷律变" (len=5) -> 预测: "，"
        ## 寒随穷律变，春逐鸟声开。
        ## 1. 寒随穷律变 -> ,
        ## 2. 随穷律变，-> 春
        ## 3. 穷律变，春 -> 逐
        ## 4. 律变，春逐 -> 鸟
        ## 5. 变，春逐鸟 -> 声
        ## 6. ，春逐鸟声 --> 开
        ##  7. 春逐鸟声开 --> 。 --> End

        ## 床前明月光，疑是地上霜
        ## 1. 床 -> ,
        ## 2.床前明月光 ->前明月光,
        ## 6. 床前明月光，疑 --> 开

        

        ## 总结： 寒随穷律变  -- > 寒随穷律变，春逐鸟声开。
        for sentence in pipeline.sentences:
            indices = pipeline.text_to_indices(sentence)
            for i in range(len(indices) - seq_len):
                # x: 当前窗口的字
                x_seq = indices[i : i + seq_len]
                # y: 下一个字
                y_char = indices[i + seq_len]
                
                self.inputs.append(x_seq)
                self.targets.append(y_char)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx]), torch.tensor(self.targets[idx])

# 配置参数
SEQ_LEN = 6   # 根据前6个字预测第7个字
BATCH_SIZE = 4 # 数据很少，batch_size 设小一点
HIDDEN_DIM = 128
EMBED_DIM = 64
EPOCHS = 200  # 数据少，需要多训练很多轮才能过拟合(记住)这些诗
LR = 0.005

dataset = PoetryDataset(pipeline, seq_len=SEQ_LEN)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# ==========================================
# 3. 模型定义 (LSTM 生成模型)
# ==========================================

## 文本 -> 文本
## 文本 -> embedding -> feature_in -> feature extractor (RNN) -> out 

class PoetryGenerator(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # 注意：这里使用单向 LSTM，bidirectional=False
        # 如果设为 True，模型无法用于生成任务（会看到未来）
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=2, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, hidden=None):
        # x: [batch, seq_len]
        embeds = self.embedding(x)      # [batch, seq, embed]
        out, hidden = self.lstm(embeds, hidden) # out: [batch, seq, hidden]
        
        # 我们只关心最后一个时间步的输出，用来预测下一个字
        # out[:, -1, :] 取序列最后一个 timestep
        last_output = out[:, -1, :] 
        
        logits = self.fc(self.dropout(last_output)) # [batch, vocab]
        return logits, hidden

# ==========================================
# 4. 训练循环
# ==========================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = PoetryGenerator(pipeline.vocab_size, EMBED_DIM, HIDDEN_DIM).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

print(f"\n开始训练 (Device: {device})...")
for epoch in range(EPOCHS):
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        # 训练时 hidden 设为 None，让它自动初始化为0

        ## model -> __call__() -> forward
        outputs, _ = model(x) 
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    if (epoch+1) % 20 == 0:
        print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss/len(dataloader):.4f}")

# ==========================================
# 5. 生成/预测逻辑
# ==========================================
def generate_poem(model, start_text, length=40, temperature=0.8):
    ## grad 关掉了
    ##使用训练阶段累积的全局均值和方差（不更新）,关闭 Dropout，所有神经元都参与计算
    model.eval()
    
    # 1. 预处理输入
    current_input = pipeline.text_to_indices(start_text)
    # 如果输入长度不够，前面补随机或者补0 (这里简单处理，假设输入够长或不做padding)
    # 实际应用中可以做 Padding，这里为了演示直接取最后 SEQ_LEN 个
    if len(current_input) > SEQ_LEN:
        current_input = current_input[-SEQ_LEN:]
    
    result = list(current_input)
    
    # 初始化 hidden state
    hidden = None
    
    with torch.no_grad():
        for _ in range(length):
            # 把当前序列转为 tensor, 增加 batch 维度 -> [1, seq_len]
            x = torch.tensor([current_input[-SEQ_LEN:]]).to(device)
            
            # 预测
            logits, hidden = model(x, hidden)
            
            # --- 采样策略 (Temperature Sampling) ---
            # temperature 越低，结果越保守（容易重复）；越高越随机
            probs = torch.softmax(logits / temperature, dim=1)
            
            # 根据概率分布抽样
            next_char_idx = torch.multinomial(probs, 1).item()
            
            # 记录结果
            result.append(next_char_idx)
            current_input.append(next_char_idx)

            ## 训练的时候加入特殊标识 <EOS> -> 可以结束了
            ## if next_char_idx == <EOS>: break
            
            
            # 如果遇到句号，其实可以根据逻辑停止，这里为了展示让它生成固定长度

    return pipeline.indices_to_text( )

# ==========================================
# 6. 测试效果
# ==========================================
print("\n=== 生成测试 ===")
# 使用数据集里的开头来测试
starts = ["寒随穷律", "晚霞聊自", "一朝春夏", "夏律昨留"]

for s in starts:
    poem = generate_poem(model, s, length=24, temperature=0.5) # 低温采样保证通顺
    print(f"开头 [{s}] -> 生成: {poem}")

print("\n=== 自由生成 (未见过的开头) ===")
print("开头 [春风] -> ", generate_poem(model, "春风", length=24, temperature=1.0))

首春:寒随穷律变，春逐鸟声开。初风飘带柳，晚雪间花梅。碧林青旧竹，绿沼翠新苔。芝田初雁去，绮树巧莺来。

初晴落景:晚霞聊自怡，初晴弥可喜。日晃百花色，风动千林翠。池鱼跃不同，园鸟声还异。寄言博通者，知予物外志。

初夏:一朝春夏改，隔夜鸟花迁。阴阳深浅叶，晓夕重轻烟。哢莺犹响殿，横丝正网天。珮高兰影接，绶细草纹连。碧鳞惊棹侧，玄燕舞檐前。何必汾阳处，始复有山泉。

度秋:夏律昨留灰，秋箭今移晷。峨嵋岫初出，洞庭波渐起。桂白发幽岩，菊黄开灞涘。运流方可叹，含毫属微理。

仪鸾殿早秋:寒惊蓟门叶，秋发小山枝。松阴背日转，竹影避风移。提壶菊花岸，高兴芙蓉池。欲知凉气早，巢空燕不窥。

秋日即目:爽气浮丹阙，秋光澹紫宫。衣碎荷疏影，花明菊点丛。袍轻低草露，盖侧舞松风。散岫飘云叶，迷路飞烟鸿。砌冷兰凋佩，闺寒树陨桐。别鹤栖琴里，离猿啼峡中。落野飞星箭，弦虚半月弓。芳菲夕雾起，暮色满房栊。

山阁晚秋:山亭秋色满，岩牖凉风度。疏兰尚染烟，残菊犹承露。古石衣新苔，新巢封古树。历览情无极，咫尺轮光暮。

帝京篇十首:秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉余。连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。惊雁落虚弦，啼猿悲急箭。阅赏诚多美，于兹乃忘倦。鸣笳临乐馆，眺听欢芳节。急管韵朱弦，清歌凝白雪。彩凤肃来仪，玄鹤纷成列。去兹郑卫声，雅音方可悦。芳辰追逸趣，禁苑信多奇。桥形通汉上，峰势接云危。烟霞交隐映，花鸟自参差。何如肆辙迹，万里赏瑶池。飞盖去芳园，兰桡游翠渚。萍间日彩乱，荷处香风举。桂楫满中川，弦歌振长屿。岂必汾河曲，方为欢宴所。落日双阙昏，回舆九重暮。长烟散初碧，皎月澄轻素。搴幌玩琴书，开轩引云雾。斜汉耿层阁，清风摇玉树。欢乐难再逢，芳辰良可惜。玉酒泛云罍，兰殽陈绮席。千钟合尧禹，百兽谐金石。得志重寸阴，忘怀轻尺璧。建章欢赏夕，二八尽妖妍。罗绮昭阳殿，芬芳玳瑁筵。佩移星正动，扇掩月初圆。无劳上悬圃，即此对神仙。以兹游观极，悠然独长想。披卷览前踪，抚躬寻既往。望古茅茨约，瞻今兰殿广。人道恶高危，虚心戒盈荡。奉天竭诚敬，临民思惠养。纳善察忠谏，明科慎刑赏。六五诚难继，四三非易仰。广待淳化敷，方嗣云亭响。

饮马长城窟行:塞外悲风切